# Tensor API

In [1]:
import torch

a = torch.ones(3, 2)
a.shape
a

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [9]:
a_t = torch.transpose(a, 0, 1)
a_t

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [10]:
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [11]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

In [12]:
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

There is no difference between the two forms; they can be used interchangeably. We mentioned the online docs earlier (http://pytorch.org/docs). They are exhaustive and well organized, with the tensor operations divided into groups:
* `Creation ops`—Functions for constructing a tensor, like `ones` and `from_numpy`
* `Indexing, slicing, joining, mutating ops`—Functions for changing the shape, stride, or content of a tensor, like `transpose` 
* `Math ops`—Functions for manipulating the content of the tensor through computations

# Tensors: Scenic views of storage

It is time for us to look a bit closer at the implementation under the hood. Values in tensors are allocated in contiguous chunks of memory managed by torch.Storage instances. A storage is a one-dimensional array of numerical data: that is, a contiguous block of memory containing numbers of a given type, such as float (32 bits representing a floating-point number) or int64 (64 bits representing an integer). A PyTorch Tensor instance is a view of such a Storage instance that is capable of indexing into that storage using an offset and per-dimension strides

![Tensor storage](images/tensor_storage.png "Tensor Storage")

## Indexing into storage

In [13]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

Even though the tensor reports itself as having three rows and two columns, the storage under the hood is a contiguous array of size 6. In this sense, the tensor just knows how to translate a pair of indices into a location in the storage. We can also index into a storage manually. For instance:

In [14]:
points_storage = points.storage()
points_storage[0]

4.0

In [15]:
points.storage()[1]

1.0

We can’t index a storage of a 2D tensor using two indices. The layout of a storage is always one-dimensional, regardless of the dimensionality of any and all tensors that might refer to it. At this point, it shouldn’t come as a surprise that changing the value of a storage leads to changing the content of its referring tensor:

In [16]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

## Modifying Storage Values

In addition to the operations on tensors introduced in the previous section, a small number of operations exist only as methods of the Tensor object. They are recognizable from a trailing underscore in their name, like zero_, which indicates that the method operates in place by modifying the input instead of creating a new output tensor and returning it. For instance, the zero_ method zeros out all the elements of the input. Any method without the trailing underscore leaves the source tensor unchanged and instead returns a new tensor

In [17]:
a = torch.ones(3, 2)
a

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [22]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

# Tensor Metadata: Size, Offset, and Stride

In order to index into a storage, tensors rely on a few pieces of information that, together with their storage, unequivocally define them: size, offset, and stride. How these interact shown in figure below. The size (or shape, in NumPy parlance) is a tuple indicating how many elements across each dimension the tensor represents. The storage offset is the index in the storage corresponding to the first element in the tensor. The stride is the number of elements in the storage that need to be skipped over to obtain the next element along each dimension.

![Tensor Metadata](images/tensor_metadata.png "Tensor Metadata")

In [4]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [19]:
second_point = points[1]
second_point

tensor([5., 3.])

In [20]:
second_point.storage_offset()

2

In [21]:
second_point.size()

torch.Size([2])

`storage_offset` get the position of the index. On the other word the resulting tensor has offset 2 in the storage (since we need to skip the first point, which has two items)

In [22]:
points.stride()

(2, 1)

## Transpose without Copying

In [23]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [24]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

We can easily verify that the two tensors share the same storage

In [25]:
id(points.storage()) == id(points_t.storage())

True

and that they differ only in shape and stride:

In [26]:
points.stride()

(2, 1)

In [27]:
points_t.stride()

(1, 2)

We can transpose `points` into `points_t`, as shown in figure below. We change the order of the elements in the stride. After that, increasing the row (the first index of the tensor) will skip along the storage by one, just like when we were moving along columns in `points`. This is the very definition of transposing. No new memory is allocated: transposing is obtained only by creating a new Tensor instance with different stride ordering than the original.